# Portfolio Optimization 

## Import Required Libraries

In [5]:
pip install yfinance

ERROR: Could not find a version that satisfies the requirement datatime (from versions: none)
ERROR: No matching distribution found for datatime


In [7]:
pip install datetime

     -------------------------------------- 52.2/52.2 kB 665.5 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install timedelta

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for timedelta: filename=timedelta-2020.12.3-py3-none-any.whl size=1556 sha256=d22327f29806c5cba100b7626dc697f84c7288c4eaa8ff3cda1b4a64ce63f953
  Stored in directory: c:\users\hardik vaibhav\appdata\local\pip\cache\wheels\0b\9a\39\614edf72f37e354917a992c40bd518b99fc9e47b73fffdee02
Successfully built timedelta
Note: you may need to restart the kernel to use updated packages.


In [10]:
import yfinance as yf 
#This library is used to pick stock prices from website of Yahoo Finance
import pandas as pd
from datetime import datetime, timedelta
#datatime allows us select a certain time range
import numpy as np
from scipy.optimize import minimize
#NumPy and SciPy will allow us to use certain statistical methods that we need

# Section 1- Define Tickers and Time Range

## Define the list of tickers

In [11]:
tickers = ['HDB','RELIANCE.NS','TCS.NS','SBIN.NS', 'AXB.IL']

## Set the end date to today

In [15]:
end_date = datetime.today()
print(end_date)

2023-11-27 19:48:44.675361


## Set the start date to 5 years ago

In [19]:
start_date = end_date - timedelta(days = 5*365)
print(start_date)

2018-11-28 19:48:44.675361


# Section 2 - Download Adjusted Closed Prices

## Create an empty DataFrame to store the adjusted close prices

In [16]:
adj_close_df = pd.DataFrame()

## Download the close prices for each ticker

In [20]:
for ticker in tickers:
    data = yf.download(ticker, start = start_date,end = end_date)
    adj_close_df[ticker] = data['Adj Close']

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


# Display the DataFrame

In [21]:
print(adj_close_df)

                  HDB  RELIANCE.NS       TCS.NS     SBIN.NS     AXB.IL
Date                                                                  
2018-11-29  49.255135  1135.725952  1759.025513  274.093018  44.678585
2018-11-30  49.095387  1134.802490  1765.933838  272.418213  44.180496
2018-12-03  48.495132  1124.013794  1778.629639  274.475830  44.130688
2018-12-04  48.064297  1119.980347  1804.154907  270.408447  43.931450
2018-12-06  49.308384  1092.133789  1787.870605  263.804962  41.739864
...               ...          ...          ...         ...        ...
2023-11-20  58.270000  2349.350098  3519.600098  563.750000  59.099998
2023-11-21  58.849998  2378.899902  3510.199951  561.500000  59.200001
2023-11-22  58.639999  2388.199951  3530.149902  558.950012  59.700001
2023-11-24  59.500000  2393.899902  3457.100098  560.349976  60.599998
2023-11-27  59.189999          NaN          NaN         NaN  60.299999

[1256 rows x 5 columns]


# Section 3 - Calculate Lognormal Returns

## Calculate the lognormal returns for each ticker

In [22]:
log_returns = np.log(adj_close_df/adj_close_df.shift(1))

## Drop any missing values

In [24]:
log_returns = log_returns.dropna()
print(log_returns)

                 HDB  RELIANCE.NS    TCS.NS   SBIN.NS    AXB.IL
Date                                                           
2018-11-30 -0.003249    -0.000813  0.003920 -0.006129 -0.011211
2018-12-03 -0.012302    -0.009553  0.007164  0.007525 -0.001128
2018-12-04 -0.008924    -0.003595  0.014249 -0.014930 -0.004525
2018-12-06  0.025554    -0.025178 -0.009067 -0.024724 -0.051174
2018-12-07 -0.013840     0.008993  0.001254 -0.000907 -0.003586
...              ...          ...       ...       ...       ...
2023-11-17  0.006038    -0.002184  0.001314 -0.037645 -0.026404
2023-11-20  0.002234    -0.002636  0.004885  0.001242 -0.011775
2023-11-21  0.009904     0.012499 -0.002674 -0.003999  0.001691
2023-11-22 -0.003575     0.003902  0.005667 -0.004552  0.008410
2023-11-24  0.014559     0.002384 -0.020910  0.002501  0.014963

[1089 rows x 5 columns]


# Section 4 - Calculate Covariance Matrix

## Calculate the covariance matrix using annualized returns

In [25]:
cov_matrix = log_returns.cov()*252
print(cov_matrix)

                  HDB  RELIANCE.NS    TCS.NS   SBIN.NS    AXB.IL
HDB          0.000467     0.000156  0.000076  0.000214  0.000319
RELIANCE.NS  0.000156     0.000370  0.000108  0.000164  0.000188
TCS.NS       0.000076     0.000108  0.000251  0.000086  0.000100
SBIN.NS      0.000214     0.000164  0.000086  0.000468  0.000350
AXB.IL       0.000319     0.000188  0.000100  0.000350  0.000724


# Section 4 - Define Portfolio Performance Metrices

## Calculate the portfolio standard deviation

This line of code calculates the portfolio variance, which is a measure of risk associated with a portfolio of assets.  

In [28]:
def standard_deviation(weights,cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)    

## Calculate the expected return

In [ ]:
def expected_return(weights,log_returns):
    return np.sum(log_returns.mean()*weights)*252

## Calculate the Sharpe Ratio

In [26]:
def sharpe_ratio(weights,log_returns, cov_matrix, risk_free_rate):
    return(expected_return(weights,log_returns)- risk_free_rate) / standard_deviation(weights,cov_matrix)

# Section 5 - Portfolio Optimization

## Set the Risk-free rate

In [29]:
risk_free_rate = 0.07 

## Define the function to minimize (negative Sharpe Ratio)

In [ ]:
def neg_sharpe_ratio(weights,log_returns, cov_matrix, risk_free_rate):
    

## Set the Constraints and Bounds 

In [ ]:
constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
bounds = [(0, 0.4) for _ in range(len(tickers))]

## Set the Initial Weights 

In [30]:
initial_weights = np.array([1/len(tickers)]*len(tickers))
print (initial_weights) 

[0.2 0.2 0.2 0.2 0.2]
